In [5]:
import requests
import os
import json
import numpy as np
import random

In [2]:
hiring_firebase = 'https://hackathon-2020-9448d.firebaseio.com/posts_data/hiring'
finding_firebase = 'https://hackathon-2020-9448d.firebaseio.com/posts_data/finding'

In [71]:
def post_to_firebase(base_link, postid, content):
#     print(content)
    r = requests.put(base_link + '/' + postid + '.json', json.dumps(content))
    return r

In [12]:
with open('./data/firebase_posts/names.txt', 'r') as names_file:
    names = names_file.read()
names = names.split('\n')
random.shuffle(names)

In [67]:
with open('./data/firebase_posts/ava_links.txt', 'r') as avatar_files:
    avas = avatar_files.read()
avas = avas.split('\n')
avas.remove('')

In [13]:
labels_dir = './data/firebase_posts/titles/'

In [17]:
#Filter the label datas
for txt_file in os.listdir(labels_dir):
    if '_label' in txt_file:
        if not os.path.exists(labels_dir + txt_file.replace('_label', '')):
            os.remove(labels_dir + txt_file)

In [27]:
label_dict = {'ban_hang': 0, 'bao_ve': 1, 'bep': 2, 'cong_nhan': 3, 'giao_vien': 4, 'giup_viec': 5, 'huong_dan_vien_du_lich': 6, 'ke_toan': 7, 'ky_su': 8, 'lai_xe': 9, 'nhan_vien': 10, 'pha_che': 11, 'phien_dich': 12, 'phuc_vu': 13, 'quan_ly': 14, 'sale_online': 15, 'tho_lap_may': 16, 'tho_xay': 17, 'thoi_vu': 18, 'tro_ly': 19, 'y_te': 20}

In [44]:
def get_hot_encode(label):
    hot_encode = [0] * len(label_dict)
    for l in label:
        hot_encode[label_dict[l]] = 1
    strr = ''
    for e in hot_encode:
        strr += str(e)
    return strr

In [36]:
class PostAttribute:
    def __init__(self, title, label, names, avas):
        self.title = title
        self.label = get_hot_encode(label)
        self.pick_info(names, avas)
        
    def pick_info(self, names, avas):
        self.name = random.choice(names)
        self.ava = random.choice(avas)

In [68]:
articles = []
for txt_file in os.listdir(labels_dir):
    if '_label' not in txt_file:
        with open(labels_dir + txt_file, 'r') as file_read:
            article = file_read.read()
            title = article
            
        with open(labels_dir + txt_file.replace('.txt', '_label.txt'), 'r') as file_read:
            label = file_read.read()
            label = label.replace(' ', '').split('\n')[:-1]
        
        articles.append(PostAttribute(title, label, names, avas))

In [42]:
def get_data_from_fb(firebase_link):
    r = requests.get(firebase_link + '.json')
    return json.loads(r.content)

finding_jobs = get_data_from_fb('https://hackathon-2020-9448d.firebaseio.com/data_matrix/finding')

In [69]:
def calculate_score(job, label):
    njob = job.split('x')[0]
    sum = 0
    for n, l in zip(njob, label):
        sum += abs(int(n) - int(l))
    return sum

In [72]:
for find_id, job in zip(finding_jobs.keys(), finding_jobs.values()):
    count = 0
    while count < 100:
        article = random.choice(articles)
        score = calculate_score(job, article.label)
        if score < 3:
            break
        else:
            count += 1
    post_to_firebase(finding_firebase, find_id, {'image': article.ava, 'name': article.name, 'title': article.title})